Dataset: https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [3]:
data ='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [4]:
!wget $data -O homework.csv 

--2025-10-20 07:55:43--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘homework.csv’

homework.csv        100%[===================>]  78.98K  --.-KB/s    in 0.07s   

2025-10-20 07:55:44 (1.04 MB/s) - ‘homework.csv’ saved [80876/80876]



In [5]:
df = pd.read_csv('homework.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [6]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [8]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [12]:
#Split categorical and numerical features
categorical = ['lead_source','industry','employment_status','location']
numerical = ['number_of_courses_viewed','annual_income','interaction_count','lead_score']

In [13]:
#Filling Missing Vales
for col in categorical:
    df[col] = df[col].fillna('NA')
for col in numerical:
    df[col] = df[col].fillna(0.0)

In [14]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

### Validation Framework

In [16]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [18]:
len(df_full_train), len(df_train), len(df_val), len(df_test), len(df)

(1169, 876, 293, 293, 1462)

### ROC AUC Score - Feature Importance

In [20]:
from sklearn.metrics import roc_auc_score

In [25]:
for col in numerical:
    auc = roc_auc_score(y_train, df_train[col])
    if auc<0.5:
        auc = roc_auc_score(y_train, -df_train[col])
    print('%s = %.3f' % (col, auc))

number_of_courses_viewed = 0.764
annual_income = 0.552
interaction_count = 0.738
lead_score = 0.614


### Training the model